Dependencies

In [ ]:
!pip install torch torchaudio tqdm soundfile


Mount Drive (Pour charger modelpth deja saved en local)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Code

In [ ]:
import os
print("Contenu du dossier /content :")
print(os.listdir("/content"))


Contenu du dossier /content :
['.config', 'drive', 'voice_dataset_final.zip', 'sample_data']


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torch.utils.data import DataLoader, Dataset
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
from tqdm import tqdm

# ========================
# ⚙️ Paramètres
# ========================
DATA_DIR = "/content/voice_data_final/voice_dataset_final"
BATCH_SIZE = 16
EPOCHS = 200
LR = 0.001
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ========================
# 📦 Dataset
# ========================
class VoiceCommandDataset(Dataset):
    def __init__(self, data_dir):
        self.data, self.labels = [], []
        self.classes = sorted(os.listdir(data_dir))
        self.mel = MelSpectrogram(sample_rate=16000, n_mels=64)
        self.db = AmplitudeToDB()
        for label, cls in enumerate(self.classes):
            cls_path = os.path.join(data_dir, cls)
            for file in os.listdir(cls_path):
                if file.endswith(".wav"):
                    self.data.append(os.path.join(cls_path, file))
                    self.labels.append(label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        waveform, sr = torchaudio.load(self.data[idx])
        waveform = torch.mean(waveform, dim=0, keepdim=True)

        # 🔹 Normalisation de la durée : 1 s = 16000 échantillons
        target_len = 16000
        if waveform.shape[1] < target_len:
            pad = target_len - waveform.shape[1]
            waveform = F.pad(waveform, (0, pad))
        else:
            waveform = waveform[:, :target_len]

        mel_spec = self.db(self.mel(waveform))
        return mel_spec, torch.tensor(self.labels[idx])


# ========================
# 🧠 Modèle CNN
# ========================
class VoiceNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )

        # Dynamically calculate the flattened size for the linear layer
        # Use a dummy input with an assumed max time_steps (e.g., 80)
        # that could be produced by collate_fn's padding.
        # This ensures the linear layer's input features match the conv output.
        with torch.no_grad():
            dummy_input_shape = (1, 1, 64, 80)  # (batch_size, channels, n_mels, time_steps)
            dummy_input = torch.zeros(dummy_input_shape)
            dummy_output = self.conv(dummy_input)
            flattened_size = dummy_output.view(dummy_output.size(0), -1).shape[1]

        self.fc = nn.Sequential(
            nn.Linear(flattened_size, 128), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [ ]:
import zipfile
import os

zip_path = "/content/voice_dataset_final.zip"   # chemin exact de ton fichier
extract_path = "/content/voice_data_final"      # dossier de sortie

# Décompression
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset extrait vers :", extract_path)
print("📂 Contenu :", os.listdir(extract_path))


✅ Dataset extrait vers : /content/voice_data_final
📂 Contenu : ['voice_dataset_final']


Padding auto pour avoir les memes longueurs de spectogrammes

In [ ]:
import torch.nn.functional as F

def collate_fn(batch):
    specs, labels = zip(*batch)
    # Trouver la taille max dans le batch
    max_len = max([s.shape[-1] for s in specs])
    padded_specs = []
    for s in specs:
        pad_size = max_len - s.shape[-1]
        s = F.pad(s, (0, pad_size))  # padding sur la dimension temporelle
        padded_specs.append(s)
    specs = torch.stack(padded_specs)
    labels = torch.tensor(labels)
    return specs, labels


In [ ]:
import os

!pip install torchcodec

# ========================
# ✨ Entraã¤nement
# ========================
dataset = VoiceCommandDataset(DATA_DIR)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
model = VoiceNet(num_classes=len(dataset.classes)).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# === Configuration pour la reprise de l'entraÃ¤nement ===
MODEL_SAVE_PATH = "/content/voice_model.pth"
OPTIMIZER_SAVE_PATH = "/content/voice_optimizer.pth" # Chemin pour sauvegarder l'Ã©tat de l'optimiseur

RESUME_TRAINING = True # Mettez Ã  True si vous voulez reprendre l'entraÃ¤nement
# Si RESUME_TRAINING est True, dÃ©finissez RESUME_FROM_EPOCH Ã  l'Ã©poque *SUIVANTE* Ã  entraÃ¤ner.
# Par exemple, si vous avez terminÃ© l'Ã©poque 20, mettez RESUME_FROM_EPOCH = 20 pour commencer l'entraÃ¤nement de l'Ã©poque 21.
RESUME_FROM_EPOCH = 20 # Exemple: Reprendre Ã  partir de l'Ã©poque 21 (aprÃ¨s l'Ã©poque 20)

start_epoch = 0 # L'Ã©poque de dÃ©but par dÃ©faut si l'on ne reprend pas

if RESUME_TRAINING and os.path.exists(MODEL_SAVE_PATH):
    print(f"Chargement du modÃ¨le depuis {MODEL_SAVE_PATH}...")
    model.load_state_dict(torch.load(MODEL_SAVE_PATH))
    if os.path.exists(OPTIMIZER_SAVE_PATH):
        print(f"Chargement de l'optimiseur depuis {OPTIMIZER_SAVE_PATH}...")
        optimizer.load_state_dict(torch.load(OPTIMIZER_SAVE_PATH))
        start_epoch = RESUME_FROM_EPOCH
        print(f"Reprise de l'entraÃ¤nement Ã  partir de l'Ã©poque {start_epoch + 1}...")
    else:
        print("Fichier d'Ã©tat de l'optimiseur non trouvÃ©. DÃ©marrage d'un nouvel entraÃ¤nement pour l'optimiseur.")
        start_epoch = RESUME_FROM_EPOCH # Continue from the specified epoch with a fresh optimizer
elif RESUME_TRAINING and not os.path.exists(MODEL_SAVE_PATH):
    print(f"Le modÃ¨le sauvegardÃ© ({MODEL_SAVE_PATH}) n'a pas Ã©tÃ© trouvÃ©. DÃ©marrage d'un nouvel entraÃ¤nement.")
    start_epoch = 0
else:
    print("DÃ©marrage d'un nouvel entraÃ¤nement.")
    start_epoch = 0
# ==========================================

print(f"ð§  EntraÃ¤nement sur {DEVICE} ({len(dataset)} Ã©chantillons) ...")
# La boucle commence maintenant Ã  partir de `start_epoch`
for epoch in range(start_epoch, EPOCHS):
    model.train()
    total_loss = 0
    for X, y in tqdm(loader, desc=f"Ãpoque {epoch+1}/{EPOCHS}"):
        X, y = X.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"ð¡ Epoch {epoch+1}: Loss={total_loss/len(loader):.4f}")

    # Sauvegarde de l'Ã©tat du modÃ¨le et de l'optimiseur aprÃ¨s chaque Ã©poque
    torch.save(model.state_dict(), MODEL_SAVE_PATH)
    torch.save(optimizer.state_dict(), OPTIMIZER_SAVE_PATH)
    print(f"âModÃ¨le et optimiseur sauvegardÃ©s aprÃ¨s l'Ã©poque {epoch+1}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.7 MB/s eta 0:00:00
Le modÃ¨le sauvegardÃ© (/content/voice_model.pth) n'a pas Ã©tÃ© trouvÃ©. DÃ©marrage d'un nouvel entraÃ¤nement.
ð§  EntraÃ¤nement sur cpu (1000 Ã©chantillons) ...


Ãpoque 1/200: 100%|██████████| 63/63 [00:11<00:00,  5.60it/s]


ð¡ Epoch 1: Loss=2.3951
âModÃ¨le et optimiseur sauvegardÃ©s aprÃ¨s l'Ã©poque 1


Ãpoque 2/200: 100%|██████████| 63/63 [00:09<00:00,  6.33it/s]


ð¡ Epoch 2: Loss=0.2971
âModÃ¨le et optimiseur sauvegardÃ©s aprÃ¨s l'Ã©poque 2


...

ð¡ Epoch 198: Loss=0.0000
âModÃ¨le et optimiseur sauvegardÃ©s aprÃ¨s l'Ã©poque 198


Ãpoque 199/200: 100%|██████████| 63/63 [00:11<00:00,  5.50it/s]


ð¡ Epoch 199: Loss=0.0000
âModÃ¨le et optimiseur sauvegardÃ©s aprÃ¨s l'Ã©poque 199


Ãpoque 200/200: 100%|██████████| 63/63 [00:10<00:00,  5.81it/s]

ð¡ Epoch 200: Loss=0.0000
âModÃ¨le et optimiseur sauvegardÃ©s aprÃ¨s l'Ã©poque 200


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for X, y in loader:
        X, y = X.to(DEVICE), y.to(DEVICE)
        outputs = model(X)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y.cpu().numpy())

print("\n✅ Évaluation du modèle :")
print(classification_report(all_labels, all_preds, target_names=dataset.classes))



✅ Évaluation du modèle :
              precision    recall  f1-score   support

      avance       1.00      1.00      1.00       200
      droite       1.00      1.00      1.00       200
      gauche       1.00      1.00      1.00       200
      recule       1.00      1.00      1.00       200
        stop       1.00      1.00      1.00       200

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000



Passage du modele sur cpu et en mode eval


In [ ]:
import torch

model.cpu()
model.eval()

VoiceNet(
  (conv): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=10240, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=5, bias=True)
  )
)

Vérification de la taille réelle de l’entrée


In [ ]:
x, _ = dataset[0]
print("Shape MelSpectrogram :", x.shape)


Shape MelSpectrogram : torch.Size([1, 64, 81])


(OPTIONNEL) Quantization PyTorch (INT8 CPU)

In [ ]:
import torch.quantization as tq

torch.backends.quantized.engine = "fbgemm"
model.qconfig = tq.get_default_qconfig("fbgemm")

model_prepared = tq.prepare(model, inplace=False)

# Calibration rapide (sans DataLoader)
with torch.no_grad():
    for _ in range(20):
        model_prepared(torch.randn(1, 1, 64, x.shape[-1]))

model_quantized = tq.convert(model_prepared, inplace=False)

print("✅ Modèle quantifié INT8 (CPU)")


/tmp/ipython-input-2162412526.py:6: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  model_prepared = tq.prepare(model, inplace=False)
/usr/local/lib/python3.12/dist-packages/torch/ao/quantization/observer.py:246: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyT

✅ Modèle quantifié INT8 (CPU)


EXPORT ONNX

In [ ]:
!pip install onnx onnxscript


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.0/683.0 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 13.1 MB/s eta 0:00:00


In [ ]:
dummy_input = torch.randn(1, 1, 64, x.shape[-1])

torch.onnx.export(
    model,  # FP32
    dummy_input,
    "voice_model.onnx",
    input_names=["input"],
    output_names=["logits"],
    opset_version=12,
    dynamic_axes={
        "input": {0: "batch"},
        "logits": {0: "batch"}
    }
)

print("✅ Export ONNX terminé : voice_model.onnx")


/tmp/ipython-input-1421536731.py:3: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W1215 11:16:40.401000 474 torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 12 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features


[torch.onnx] Obtain model graph for `VoiceNet([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `VoiceNet([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/__init__.py", line 127, in call
    converted_proto = _c_api_utils.call_onnx_api(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/_c_api_utils.py", line 65, in call_onnx_api
    result = func(proto)
             ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/__init__.py", line 122, in _partial_convert_version
    return onnx.version_converter.convert_version(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnx/version_converter.py", line 39, in convert_version
    converted_model_str = C.convert_version(model_str, target_version)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: /github/workspace/onnx/version_converter/BaseConverter.h:68: adapter_lookup: Assertion `false`

[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 1 of general pattern rewrite rules.
✅ Export ONNX terminé : voice_model.onnx


TESTER LE ONNX

In [ ]:
!pip install onnxruntime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.7 MB/s eta 0:00:00


In [ ]:
import onnxruntime as ort
import numpy as np

sess = ort.InferenceSession("voice_model.onnx")
y = sess.run(
    None,
    {"input": dummy_input.numpy()}
)

print("ONNX output shape :", y[0].shape)


ONNX output shape : (1, 5)


Téléchargement de files

In [ ]:
from google.colab import files

files.download("voice_model.pth")
files.download("voice_model.onnx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
torch.onnx.export(
    model,
    dummy_input,
    "voice_model_single.onnx",
    input_names=["input"],
    output_names=["logits"],
    opset_version=12,
    dynamic_axes={
        "input": {0: "batch"},
        "logits": {0: "batch"}
    },
    use_external_data_format=False  # 🔴 IMPORTANT
)


NameError: name 'torch' is not defined